# PHR HCS calculation

How the heck to calculate this HCS with Clash ?

First of all
- It's a CRC-8 but not a standard one
- It can be run in two parts : A and B. Only B depends on the input and is "only" a polynomial division. It's then a simple modulo addition

The goal is to find a "circuit-like" approach to the B calculation.

What has worked ? Making a simulator that uses two matrices :
- X : calculate the next state based on the current state (how each register affects the other through xor gates)
- B : How the input affects each register

Run the simulator one step at a time manually to detect where a xor gate has failed. Then the gate is corrected (either in X or B) and run again

When the simulator looks correct, it is automatically tested with ~10'000 different messages against the HCS calculation

## Conclusion

It worked ! the B calculation is an 8 bit shift register with those equations (for each step):

```python
# b is the input

m0 = b ^ m7
m1 = b ^ m7 ^ m0
m2 = b ^ m7 ^ m1
m3 = m2
m4 = m3
m5 = m4
m6 = m5
m7 = m6

B = [m7, m6, m5, m4, m3, m2, m1, m0]
```





In [4]:
import numpy as np
from numpy import poly1d, polymul, polydiv, polyadd

In [160]:
def toHex(arr):
    listOutput = np.array([int(f'0b{"".join([str(y) for y in x])}', 2) for x in np.reshape(arr, (-1,8))])

    if listOutput.size == 1:
        return listOutput[0]
    else:
        return listOutput


def HCS_calculation(input_array):
    """
    Applies HCS crc to input array
    See 18.2.1.3 PHR

    Parameters
    ----------
    input_array : ndarray
        Input values (0 and 1) in a numpy array. [0] is the first element
    
    Returns
    -------
    output_array : ndarray
        Calculated CRC
    """
    # Generator polynomial x^8 + x^2 + x + 1
    G = poly1d([1, 0, 0, 0, 0, 0, 1, 1, 1])
    # part a)
    k = poly1d(np.block([1, np.zeros(22)]))
    b = poly1d(np.ones(8))
    c = polymul(k, b)
    _, A = polydiv(c, G)
    A = np.mod(A, 2).astype(int)
    # part b)
    a = polymul(poly1d(input_array), poly1d(np.block([1, np.zeros(8)])))
    _, B = polydiv(a, G)
    # Pad B to have the same size as A (this has been tested and is identical)
    if B.order + 1 < 8:
        B = np.block([np.zeros(8 - (len(B) + 1)), B])
    B = np.mod(B, 2).astype(int)

    # one's complement of the modulo-2 sum    
    output_array = 1 - np.mod(polyadd(A, B), 2)
    return A, B, output_array

In [170]:
inputs = [
    [],
    [1],
    [0]
]

for i in inputs:
    a = np.array(i)
    A, B, O = HCS_calculation(a)
    print(f"input = {i}")
    print(f"  A = {A} ({toHex(A):02X})")
    print(f"  B = {B}")
    print(f"  O = {O} ({toHex(O):02X})")

input = []
  A = [0 1 0 0 1 0 1 1] (4B)
  B = [0 0 0 0 0 0 0 0]
  O = [1 0 1 1 0 1 0 0] (B4)
input = [1]
  A = [0 1 0 0 1 0 1 1] (4B)
  B = [0 0 0 0 0 1 1 1]
  O = [1 0 1 1 0 0 1 1] (B3)
input = [0]
  A = [0 1 0 0 1 0 1 1] (4B)
  B = [0 0 0 0 0 0 0 0]
  O = [1 0 1 1 0 1 0 0] (B4)


In [258]:
inputs = [
    [0],
    [0,0],
    [0,1],
    [1],
    [1,0],
    [1,0,0],
    [1,0,0,0],
    [1,0,0,0,0],
    [1,0,0,0,0,0],
    [1,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0,0,1],
]

print(f"{'i':>30} -> B")
for i in inputs:
    a = np.array(i)
    A, B, O = HCS_calculation(a)
    print(f"{str(i):>30} -> {B}")

                             i -> B
                           [0] -> [0 0 0 0 0 0 0 0]
                        [0, 0] -> [0 0 0 0 0 0 0 0]
                        [0, 1] -> [0 0 0 0 0 1 1 1]
                           [1] -> [0 0 0 0 0 1 1 1]
                        [1, 0] -> [0 0 0 0 1 1 1 0]
                     [1, 0, 0] -> [0 0 0 1 1 1 0 0]
                  [1, 0, 0, 0] -> [0 0 1 1 1 0 0 0]
               [1, 0, 0, 0, 0] -> [0 1 1 1 0 0 0 0]
            [1, 0, 0, 0, 0, 0] -> [1 1 1 0 0 0 0 0]
         [1, 0, 0, 0, 0, 0, 0] -> [1 1 0 0 0 1 1 1]
      [1, 0, 0, 0, 0, 0, 0, 0] -> [1 0 0 0 1 0 0 1]
   [1, 0, 0, 0, 0, 0, 0, 0, 0] -> [0 0 0 1 0 1 0 1]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 1] -> [0 0 1 0 1 1 0 1]


In [259]:
M = np.array([
    [0,0,0,0,0,0,0,1], # * xor = m0
    [1,0,0,0,0,0,0,1], # * xor = m1
    [0,1,0,0,0,0,0,1], # * xor = m2
    [0,0,1,0,0,0,0,0], # * xor = m3
    [0,0,0,1,0,0,0,0], # * xor = m4
    [0,0,0,0,1,0,0,0], # * xor = m5
    [0,0,0,0,0,1,0,0], # * xor = m6
    [0,0,0,0,0,0,1,0], # * xor = m7
])

B = np.array([
    1, # -> m0
    1, # -> m1
    1, # -> m2
    0, # -> m3
    0, # -> m4
    0, # -> m5
    0, # -> m6
    0, # -> m7
])
# M[0,2] -> m2 in XOR for m0

def step(M, B, x, b):
    return np.mod(M @ x + B * b, 2)

inputs = [1, 0, 0, 0, 0, 0, 0, 0, 0, 1]
x0 = np.array([0,0,0,0,0,0,0,0])
x = x0.copy()
for b in inputs:
    x = step(M, B, x, b)
    print(f"{b} -> {x[::-1]}")

1 -> [0 0 0 0 0 1 1 1]
0 -> [0 0 0 0 1 1 1 0]
0 -> [0 0 0 1 1 1 0 0]
0 -> [0 0 1 1 1 0 0 0]
0 -> [0 1 1 1 0 0 0 0]
0 -> [1 1 1 0 0 0 0 0]
0 -> [1 1 0 0 0 1 1 1]
0 -> [1 0 0 0 1 0 0 1]
0 -> [0 0 0 1 0 1 0 1]
1 -> [0 0 1 0 1 1 0 1]


In [269]:
for _ in range(10_000):
    message = np.random.randint(0,2,22)
    _, hcs, _ = HCS_calculation(message)
    new = np.zeros(8)
    for m in message:
        new = step(M, B, new, m)
    assert np.array_equal(new[::-1], hcs)


In [141]:
inputs = []
outputs = []

for _ in range(10):
    inp = np.random.randint(0,2,8)
    out = HCS_calculation(inp)[-1]
    outStr = ''.join([str(x) for x in out])

    inputs.append(inp)
    outputs.append(out)

outputs = np.stack(outputs, axis=1).T
inputs = np.stack(inputs, axis=1).T

print(f"inputs : {inputs.shape}")
print(f"outputs : {outputs.shape}")

inputs : (10, 8)
outputs : (10, 8)


In [ ]:
xormatrix = np.









In [142]:
print(outputs)
print(inputs)
# Check if the last digit of the output is always the same as the last digit of the input
print(inputs[:,-1])
print(outputs[:,-1])

[[0 0 1 1 1 1 1 0]
 [0 0 1 0 0 0 1 0]
 [0 1 0 0 0 0 1 1]
 [1 0 0 1 1 1 0 1]
 [1 0 0 1 0 1 1 1]
 [0 1 0 0 1 1 1 0]
 [0 0 1 1 0 0 1 0]
 [0 0 1 1 1 1 1 1]
 [0 1 1 0 1 1 0 0]
 [1 1 1 1 0 1 0 1]]
[[1 1 1 0 1 1 0 0]
 [1 1 1 0 1 0 0 0]
 [1 0 0 1 0 0 1 0]
 [0 1 1 0 0 0 1 0]
 [0 0 0 0 1 1 0 1]
 [1 1 1 1 1 1 0 0]
 [0 1 0 1 1 0 1 1]
 [0 0 1 1 0 1 0 1]
 [0 0 1 0 1 0 0 0]
 [0 0 0 1 1 0 1 1]]
[0 0 0 0 1 0 1 1 0 1]
[0 0 1 1 1 0 0 1 0 1]
